# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [7]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [8]:
# constants

MODEL_GPT = 'gpt-4.1'
MODEL_LLAMA = 'llama3.2'

In [9]:
# set up environment

load_dotenv()
openai = OpenAI()

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
# Get gpt-4.1 to answer, with streaming

# Create prompts
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

# Create messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Stream the response from OpenAI
print("🤖 GPT-4.1 Response:")
print("-" * 50)

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

🤖 GPT-4.1ponse:
--------------------------------------------------


Certainly! Let's break down the code and explain its function and reasoning:

python
yield from {book.get("author") for book in books if book.get("author")}


## 1. **Understanding the Context**

Assume `books` is an iterable (like a list) of dictionaries, with each dictionary representing a book. Each dictionary might have an `"author"` key.

Example:
python
books = [
    {"title": "1984", "author": "George Orwell"},
    {"title": "Python 101", "author": "Michael Driscoll"},
    {"title": "Book with no author"}
]


---

## 2. **Breaking Down the Code**

### **A. Set Comprehension**
python
{book.get("author") for book in books if book.get("author")}

- **Purpose:** Builds a set of all `"author"` values found in the `books` list—ignoring any books without an `"author"` (i.e., where `book.get("author")` is `None` or falsy).
- **How it works:**
  * `book.get("author")`: Tries to fetch the "author" value; returns `None` if not present.
  * `if book.get("author")`: Includes only those books with a non-empty, truthy "author".
  * The `{ ... for ... if ...}` makes a **set comprehension**. Sets automatically filter out duplicates—so you'll get **unique author names** only.

#### Example Output:
python
{"George Orwell", "Michael Driscoll"}

(Assuming the sample books above.)

---

### **B. `yield from`**

- `yield from <iterable>` is a generator statement in Python.
- It **delegates** yielding items *from the supplied iterable* (in this case, the set of authors) to the "outer" generator.
- This means it will **yield each unique author name**, one at a time.

---

## 3. **What Does the Whole Line Do?**

- This line is intended to be used **inside a generator function**. (It won’t work at the top level.)
- It collects *all unique (non-empty) author names* in `books`, and yields them one at a time.
- Usage in a context:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


**Usage Example:**
python
for author in unique_authors(books):
    print(author)
# Output:
# George Orwell
# Michael Driscoll


*(Order is arbitrary because sets are unordered)*

---

## 4. **Why Use This Approach?**

- **Set comprehension**: Ensures authors are unique; no duplicates.
- **Filtering**: The `if book.get("author")` check skips missing or empty author names.
- **Generator pattern**: Delivers results lazily, one at a time, for memory efficiency.

---

## **Summary Table**

| Part                                | Purpose                        |
|--------------------------------------|--------------------------------|
| `{book.get("author") ...}`           | Collect unique author names    |
| `if book.get("author")`              | Skip missing/empty authors     |
| `yield from ...`                     | Yield each author (generator)  |

---

## **In Short**

This code generates (one at a time) all unique, non-empty author names found in a `books` iterable, skipping any book with a missing or empty author. It does this efficiently and concisely, leveraging set comprehensions and Python's generator features.

In [12]:
# Get Llama 3.2 to answer

print("\n\n🦙 Llama 3.2 Response:")
print("-" * 50)

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))



🦙 Llama 3.2 Response:
--------------------------------------------------


**Generator Expression Explanation**

The given code snippet is a Python expression that uses a combination of generator expressions, `yield from`, and dictionary methods to extract authors from a list of books. Here's a breakdown:

1. `{book.get("author") for book in books if book.get("author")}`:
   - This is a **generator expression**, which is similar to a regular list comprehension but returns an iterator instead.
   - The expression iterates over the `books` list and yields values for each item that passes the condition `if book.get("author")`.
   - If an item doesn't have a key named "author", it will be skipped.

2. `yield from {book.get("author") ...}`:
   - This is where things get interesting.
   - The `yield from` keyword in Python 3.3+ allows you to delegate the iteration over another iterable (in this case, the generator expression).
   - When `yield from` is encountered, the current function pauses and yields values from the specified iterable.

So, when we put it all together:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This code snippet essentially creates a new iterator that produces authors one by one. For each book in `books` where an author is found, its value (or the author's name) will be yielded.

**Why use this?**

Using generator expressions and `yield from` achieves several benefits:

*   Memory efficiency: Instead of storing all the values in memory at once, we create an iterator that yields one item at a time. This makes it suitable for handling large datasets.
*   Flexibility: Generator expressions can be used to perform more complex transformations on data than regular list comprehensions or for loops.

**Example Use Case**

Suppose you're building an application where you need to process a large number of books and their authors. You could use the provided code snippet as part of your processing pipeline:

```python
def extract_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Sample usage:
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

for author in extract_authors(books):
    print(author)
```

In this example, `extract_authors` returns an iterator that yields authors one by one. The `print` statement demonstrates how to iterate over the returned values.

## 🎉 Congratulations!

You've successfully built a technical question answering tool using both OpenAI and Ollama!

### Key Features Implemented:
- ✅ OpenAI GPT-4.1 integration with streaming responses
- ✅ Ollama Llama 3.2 integration
- ✅ Real-time response display with markdown formatting
- ✅ Professional system prompts for technical tutoring

### Next Steps:
- Try changing the `question` variable to ask different technical questions
- After Week 2, you'll be able to add a User Interface to this tool
- Consider adding interactive input: `my_question = input("Please enter your question:")`
